# Script for a period

In [18]:
import requests
import json
import pandas as pd
from datetime import date
from clickhouse_connect import get_client
from datetime import datetime, timedelta
from dotenv import load_dotenv
import os
import time


load_dotenv()

True

In [19]:
KeyGuten = os.getenv('KeyGuten')
KeyGiper = os.getenv('KeyGiper')
KeyKitchen = os.getenv('KeyKitchen')
KeySmart = os.getenv("KeySmart")

## Списки кампаний

Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1count/get

In [20]:


# API endpoint
url = 'https://advert-api.wildberries.ru/adv/v1/promotion/count'

# Headers including the API key for authentication
headers_guten = {
    'Authorization': KeyGuten,
    'Accept': 'application/json'
}

headers_giper = {
    'Authorization': KeyGiper,
    'Accept': 'application/json'
}

headers_kitchen = {
    'Authorization': KeyKitchen,
    'Accept': 'application/json'
}

headers_smart = {
    'Authorization': KeySmart,
    'Accept': 'application/json'
}

# Make the GET request
response_guten = requests.get(url, headers=headers_guten)
response_giper = requests.get(url, headers=headers_giper)
response_kitchen = requests.get(url, headers=headers_kitchen)
response_smart = requests.get(url, headers=headers_smart)

# Check if the request was successful
if response_guten.status_code == 200 and response_giper.status_code == 200 and response_kitchen.status_code == 200 and response_smart.status_code == 200:
    # Parse the JSON response
    data_guten = response_guten.json()
    data_giper = response_giper.json()
    data_kitchen = response_kitchen.json()
    data_smart = response_smart.json()
    print("Data retrieved successfully")
else:
    print(f"Failed to retrieve data. Status code: {response_guten.status_code,response_giper.status_code,response_giper.status_code,response_smart.status_code }")
    print(f"Response: {response_guten.text, response_giper.text,response_giper.text,response_smart.text }")

Data retrieved successfully


In [21]:
data_giper

{'adverts': [{'type': 8,
   'status': 7,
   'count': 7,
   'advert_list': [{'advertId': 17741906,
     'changeTime': '2024-06-08T10:38:13.264875+03:00'},
    {'advertId': 17748698, 'changeTime': '2024-06-08T10:42:37.987864+03:00'},
    {'advertId': 17749089, 'changeTime': '2024-06-08T10:41:42.952579+03:00'},
    {'advertId': 17749228, 'changeTime': '2024-06-24T11:12:43.162712+03:00'},
    {'advertId': 17749561, 'changeTime': '2024-06-08T10:38:39.558529+03:00'},
    {'advertId': 23227483, 'changeTime': '2025-02-19T13:01:41.621239+03:00'},
    {'advertId': 23299304, 'changeTime': '2025-02-19T13:01:31.542622+03:00'}]},
  {'type': 9,
   'status': 11,
   'count': 5,
   'advert_list': [{'advertId': 22844853,
     'changeTime': '2025-02-27T14:07:48.633319+03:00'},
    {'advertId': 22844893, 'changeTime': '2025-01-31T13:31:32.654625+03:00'},
    {'advertId': 22950974, 'changeTime': '2025-03-03T12:35:20.701855+03:00'},
    {'advertId': 22951078, 'changeTime': '2025-03-03T12:47:32.279379+03:00'}

In [22]:
# Flatten the JSON data
df_guten = pd.json_normalize(
    data_guten['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_giper = pd.json_normalize(
    data_giper['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_kitchen = pd.json_normalize(
    data_kitchen['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)
df_smart = pd.json_normalize(
    data_smart['adverts'],
    record_path='advert_list', 
    meta=['type', 'status', 'count']
)

# Convert 'changeTime' to datetime format
df_guten['changeTime'] = pd.to_datetime(df_guten['changeTime'])
df_giper['changeTime'] = pd.to_datetime(df_giper['changeTime'])
df_kitchen['changeTime'] = pd.to_datetime(df_kitchen['changeTime'])
df_smart['changeTime'] = pd.to_datetime(df_smart['changeTime'])

# Filter rows where the year of 'changeTime' is 2024 or later
#df_guten = df_guten[df_guten['changeTime'].dt.year >= 2024]
#df_giper = df_giper[df_giper['changeTime'].dt.year >= 2024]
#df_kitchen = df_kitchen[df_kitchen['changeTime'].dt.year >= 2024]
#df_smart = df_smart[df_smart['changeTime'].dt.year >= 2024]

# Reset the index and drop the old index
df_guten = df_guten.reset_index(drop=True)
df_giper = df_giper.reset_index(drop=True)
df_kitchen = df_kitchen.reset_index(drop=True)
df_smart = df_smart.reset_index(drop=True)

# Display the updated DataFrame
#df_guten
#df_giper
#df_kitchen

In [23]:
#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

### Информация о кампаниях


Link: https://dev.wildberries.ru/openapi/promotion#tag/Kampanii/paths/~1adv~1v1~1promotion~1adverts/post

In [24]:
# Create chunks of 50 campaign IDs each
chunk_size = 50
campaign_chunks_guten = [df_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_guten), chunk_size)]
campaign_chunks_giper = [df_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_giper), chunk_size)]
campaign_chunks_kitchen = [df_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_kitchen), chunk_size)]
campaign_chunks_smart = [df_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(df_smart), chunk_size)]

# Print the list of all chunks
print("Campaign Chunks for WB-GutenTech:", campaign_chunks_guten)
print("Campaign Chunks for WB-ГиперМаркет:", campaign_chunks_giper)
print("Campaign Chunks for WB-KitchenAid:", campaign_chunks_kitchen)
print("Campaign Chunks for WB-Smart-Market:", campaign_chunks_smart)

# Define the API endpoint
url = "https://advert-api.wildberries.ru/adv/v1/promotion/adverts"

# Define the query parameters
query_params = {
    "order": "create",  # Order by the "change" field
    "direction":"desc"
}


# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, project_name):
    all_campaign_data = []
    for idx, chunk in enumerate(chunks):
        response = requests.post(url, params=query_params, json=chunk, headers=headers)
        if response.status_code == 200:
            data = response.json()
            all_campaign_data.extend(data)
            print(f"Data retrieved successfully for {project_name} chunk {idx + 1}")
        else:
            print(f"Error for {project_name} chunk {idx + 1}: {response.status_code}, {response.text}")
        time.sleep(1)  # Add a delay to avoid hitting API rate limits
    return all_campaign_data

# Fetch campaign data for each project
all_campaign_data_guten = fetch_campaign_data(campaign_chunks_guten, headers_guten, 'WB-GutenTech')
all_campaign_data_giper = fetch_campaign_data(campaign_chunks_giper, headers_giper, 'WB-ГиперМаркет')
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks_kitchen, headers_kitchen, 'WB-KitchenAid')
all_campaign_data_smart = fetch_campaign_data(campaign_chunks_smart, headers_smart, 'WB-Smart-Market')

# Create DataFrames from the campaign data
campaigns_guten = pd.DataFrame(all_campaign_data_guten)
campaigns_giper = pd.DataFrame(all_campaign_data_giper)
campaigns_kitchen = pd.DataFrame(all_campaign_data_kitchen)
campaigns_smart = pd.DataFrame(all_campaign_data_smart)

# Sort each DataFrame by 'createTime' in descending order
campaigns_guten = campaigns_guten.sort_values(by='createTime', ascending=False)
campaigns_giper = campaigns_giper.sort_values(by='createTime', ascending=False)
campaigns_kitchen = campaigns_kitchen.sort_values(by='createTime', ascending=False)
campaigns_smart = campaigns_smart.sort_values(by='createTime', ascending=False)

campaigns_guten['Project'] = 'WB-GutenTech'
campaigns_giper['Project'] = 'WB-ГиперМаркет'
campaigns_kitchen['Project'] = 'WB-KitchenAid'
campaigns_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
combined_campaigns = pd.concat([campaigns_guten, campaigns_giper, campaigns_kitchen,campaigns_smart], ignore_index=True)

combined_campaigns['Marketplace'] = 'Wildberries'

print("Combined Campaign Data")
#campaigns_kitchen

Campaign Chunks for WB-GutenTech: [[2851788, 5146621, 21639902, 22165389, 22255058, 7202847, 19276556, 19276694, 19276746, 21202745, 21203007, 21203075, 21203170, 21204098, 21204193, 21204224, 21204285, 21212715, 21965547, 7202795, 8004530, 8743562, 9496209, 9496385, 9591336, 11296204, 11296333, 16255090, 16301721, 16301819, 16912359, 16912972, 16913380, 16913696, 16913774, 16914258, 16917733, 16917791, 16918232, 16918415, 16919369, 16920802, 16923161, 16923906, 16923933, 16924292, 17266442, 17563748, 17563873, 17563938], [17687135, 17687241, 17687293, 17687357, 17688586, 17688635, 18640342, 18870731, 18870828, 19262099, 19263710, 19263824, 19265149, 19265205, 19626785, 19662812, 19904919, 19994021, 20039654, 20139599, 20175678, 20334946, 20354793, 20397195, 21040251, 21040485, 21040635, 21040756, 21040856, 21040946, 21385613, 21386362, 21482495, 21482904, 21498202, 8618594, 8743328, 7202878, 7783591, 7783632, 7783708, 7783759, 7784050, 213920, 213946, 213954, 225156, 232543, 232547, 2

In [25]:
# Keep only the desired columns
columns_to_keep = ["endTime", "createTime", "startTime","name", "advertId", "status", "type","Project","Marketplace"]
# Ensure filtered_df is a copy of the slice, not a view
filtered_df = combined_campaigns[columns_to_keep].copy()
filtered_df['endTime'] = pd.to_datetime(filtered_df['endTime'], format='mixed').dt.date
filtered_df['createTime'] = pd.to_datetime(filtered_df['createTime'], format='mixed').dt.date
filtered_df['startTime'] = pd.to_datetime(filtered_df['startTime'], format='mixed').dt.date

# Mapping dictionaries for 'status' and 'type'
status_mapping = {
    -1: "Кампания в процессе удаления",
    4: "Готова к запуску",
    7: "Кампания завершена",
    8: "Отказался",
    9: "Идут показы",
    11: "Кампания на паузе"
}

type_mapping = {
    4: "Кампания в каталоге (устаревший тип)",
    5: "Кампания в карточке товара (устаревший тип)",
    6: "Кампания в поиске (устаревший тип)",
    7: "Кампания в рекомендациях на главной странице (устаревший тип)",
    8: "Автоматическая кампания",
    9: "Аукцион"
}

# Replace numeric values with their string descriptions
filtered_df['status'] = filtered_df['status'].replace(status_mapping)
filtered_df['type'] = filtered_df['type'].replace(type_mapping)

# Display the updated DataFrame
filtered_df

,endTime,createTime,startTime,name,advertId,status,type,Project,Marketplace
0,2100-01-01,2025-02-19,2025-02-19,Mr Proper_19.02.2025,23408417,Кампания на паузе,Автоматическая кампания,WB-GutenTech,Wildberries
1,2100-01-01,2025-02-12,2025-03-14,Venus_12.02.2025,23231292,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
2,2100-01-01,2025-02-12,2025-03-10,Always_12.02.2025,23230882,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
3,2100-01-01,2025-01-27,2025-01-27,AND_Тонометры на запястье_27.01.2025,22846531,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
4,2100-01-01,2025-01-27,2025-03-12,AND_Тонометры авто_27.01.2025,22846516,Идут показы,Автоматическая кампания,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...
470,2024-06-23,2024-03-21,2024-04-18,Насадки Oral-B Автореклама,15562382,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries
471,2100-01-01,2024-03-21,2025-02-20,Braun_Электробритвы_22.10.2024,15562363,Идут показы,Автоматическая кампания,WB-Smart-Market,Wildberries
472,2024-10-11,2024-03-21,2024-07-02,Электрические зубные щетки Oral-B Поиск + Каталог,15562321,Кампания завершена,Аукцион,WB-Smart-Market,Wildberries
473,2024-10-15,2024-03-21,2024-08-27,Эпиляторы Braun_7 SKU_27.08.2024,15562282,Кампания завершена,Автоматическая кампания,WB-Smart-Market,Wildberries


## Статистика кампаний


Link: https://dev.wildberries.ru/openapi/analytics#tag/Statistika-po-prodvizheniyu/paths/~1adv~1v2~1fullstats/post

Данные вернутся для кампаний в статусах:

* 7 — завершена
* 9 — идут показы
* 11 — пауза из-за расхода бюджета

Response Schema: 

* dates	/ Array of strings <date> [ items <date > ]/ Даты, за которые необходимо выдать информацию.
* views	/ integer/ Количество просмотров./ За все дни, по всем артикулам WB и платформам.
* clicks	/ integer/ Количество кликов./ За все дни, по всем артикулам WB и платформам.
* ctr	/ number/ Показатель кликабельности./ Отношение числа кликов к количеству показов. Выражается в процентах./ За все дни, по всем артикулам WB и платформам.
* cpc	/ number/ Средняя стоимость клика, ₽./ За все дни, по всем артикулам WB и платформам.
* sum	/ number/ Затраты, ₽./ За все дни, по всем артикулам WB и платформам.
* atbs	/ integer/ Количество добавлений товаров в корзину./ За все дни, по всем артикулам WB и платформам.
* orders	/ integer/ Количество заказов./ За все дни, по всем артикулам WB и платформам.
* cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании./ За все дни, по всем артикулам WB и платформам.
* shks	/ integer/ Количество заказанных товаров, шт./ За все дни, по всем артикулам WB и платформам.
* sum_price	/ number/ Заказов на сумму, ₽/ За все дни, по всем артикулам WB и платформам.
* days	/ Array of objects (Days)/ Статистка по дням
    Array 
    * date	/ string <date-time>/ Дата, за которую представлены данные/ 
    * views	/ integer/ Количество просмотров/ 
    * clicks	/ integer/ Количество кликов/ 
    * ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
    * cpc	/ number/ Средняя стоимость клика, ₽./ 
    * sum	/ number/ Затраты, ₽./ 
    * atbs	/ integer/ Количество добавлений товаров в корзину/ 
    * orders	/ integer/ Количество заказов/ 
    * cr	/ integer/ CR(conversion rate) — отношение количества заказов к общему количеству посещений кампании/ 
    * shks	/ integer/ Количество заказанных товаров, шт./ 
    * sum_price	/ number/ Заказов на сумму, ₽/ 
    * apps	/ Array of objects/ Блок информации о платформе
    Array 
        - views	/ integer/ Количество просмотров/ 
        - clicks	/ integer/ Количество кликов/ 
        - ctr	/ number/ Показатель кликабельности, отношение числа кликов к количеству показов, %/ 
        - cpc	/ number/ Средняя стоимость клика, ₽./ 
        - sum	/ number/ Затраты, ₽./ 
        - atbs	/ integer/ Количество добавлений товаров в корзину/ 
        - orders	/ integer/ Количество заказов/ 
        - cr	/ integer/ CR(conversion rate) — это отношение количества заказов к общему количеству посещений кампании/ 
        - shks	/ integer/ Количество заказанных товаров, шт./ 
        - sum_price	/ number/ Заказов на сумму, ₽/ 
        - nm	/ Array of objects/ Блок статистики по артикулам WB/ 
        - appType	/ integer/ Тип платформы (1 - сайт, 32 - Android, 64 - IOS)
* boosterStats	/ Array of objects (BoosterStats)/ Статистика по средней позиции товара на страницах поисковой выдачи и каталога (для автоматических кампаний).
* advertId	/ integer/ ID кампании

In [ ]:
# API endpoint
url = "https://advert-api.wildberries.ru/adv/v2/fullstats"

# Define the date interval
date_interval = {
    "begin": "2025-03-13",  # Replace with your start date
    "end": "2025-03-13"     # Replace with your end date
}

# Create chunks of 100 campaign IDs each
chunk_size = 100

### Guten

In [27]:
# Assume campaigns_guten DataFrame is already defined and contains 'advertId'
# Create chunks of 100 campaign IDs each
campaign_chunks = [campaigns_guten['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_guten), chunk_size)]

# List to store all campaign data
all_campaign_data_guten = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_guten = fetch_campaign_data(campaign_chunks, headers_guten, date_interval)
# Combine all campaign data into a DataFrame
campaign_guten_df = pd.json_normalize(all_campaign_data_guten)

# Save the combined data to a text file
with open("response_data_guten.txt", "w", encoding="utf-8") as file:
    json.dump(all_campaign_data_guten, file, indent=4, ensure_ascii=False)

# Display the combined DataFrame
print("Combined Campaign Data")

Data retrieved successfully for chunk 1
Data retrieved successfully for chunk 2
Error for chunk 3: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 4: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Error for chunk 5: 400, {"error":"Invalid Params: there are no companies with correct intervals"}
Combined Campaign Data


### Giper

In [28]:
campaign_chunks = [campaigns_giper['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_giper), chunk_size)]

# List to store all campaign data
all_campaign_data_giper = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data
# Fetch campaign data
all_campaign_data_giper = fetch_campaign_data(campaign_chunks, headers_giper, date_interval)

Data retrieved successfully for chunk 1


### Kitchen

In [29]:
chunk_size = 100
campaign_chunks = [campaigns_kitchen['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_kitchen), chunk_size)]

# List to store all campaign data
all_campaign_data_kitchen = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_kitchen = fetch_campaign_data(campaign_chunks, headers_kitchen, date_interval)


Data retrieved successfully for chunk 1


### Smart

In [30]:
campaign_chunks = [campaigns_smart['advertId'][i:i + chunk_size].tolist() for i in range(0, len(campaigns_smart), chunk_size)]

# List to store all campaign data
all_campaign_data_smart = []

# Function to fetch campaign data
def fetch_campaign_data(chunks, headers, date_interval):
    all_data = []
    for idx, chunk in enumerate(chunks):
        # Construct the payload for the current chunk
        payload = [{"id": campaign_id, "interval": date_interval} for campaign_id in chunk]

        # Send the POST request
        response = requests.post(url, headers=headers, json=payload)

        # Add a delay to avoid hitting API rate limits
        time.sleep(65)

        # Check the response status
        if response.status_code == 200:
            # Parse the JSON response
            data = response.json()
            all_data.extend(data)
            print(f"Data retrieved successfully for chunk {idx + 1}")
        else:
            print(f"Error for chunk {idx + 1}: {response.status_code}, {response.text}")
    return all_data

# Fetch campaign data
all_campaign_data_smart = fetch_campaign_data(campaign_chunks, headers_smart, date_interval)

Data retrieved successfully for chunk 1


In [31]:
campaign_guten_df

,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,days,boosterStats,advertId,interval.begin,interval.end
0,11223,407,3.63,3.41,1389.23,18,3,0.74,3,6615,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 181893...",21498270,2025-03-01,2025-03-12
1,42800,950,2.22,9.93,9428.89,69,10,1.05,10,83743,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",NaN,21204193,2025-03-01,2025-03-12
2,17,1,5.88,1.36,1.36,0,0,0.00,0,0,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 263764...",20397161,2025-03-01,2025-03-12
3,55328,1972,3.56,22.16,43704.71,199,43,2.18,43,142942,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",NaN,21965547,2025-03-01,2025-03-12
4,11445,708,6.19,2.59,1831.20,65,9,1.27,9,43633,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...",NaN,21204098,2025-03-01,2025-03-12
5,1,0,0.00,0.00,0.09,0,0,0.00,0,0,"[{'date': '2025-03-11T03:00:00+03:00', 'views'...",NaN,21698516,2025-03-01,2025-03-12
6,9980,350,3.51,5.54,1937.72,4,0,0.00,0,0,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 161420...",20487381,2025-03-01,2025-03-12
7,1,0,0.00,0.00,0.03,0,0,0.00,0,0,"[{'date': '2025-03-02T03:00:00+03:00', 'views'...",NaN,21624823,2025-03-01,2025-03-12
8,28061,736,2.62,8.46,6229.30,109,29,3.94,30,42830,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 266556...",21821164,2025-03-01,2025-03-12
9,18564,726,3.91,2.24,1624.94,49,6,0.83,6,116089,"[{'date': '2025-03-01T03:00:00+03:00', 'views'...","[{'date': '2025-03-01T00:00:00Z', 'nm': 147720...",21202453,2025-03-01,2025-03-12


In [32]:
print(campaign_guten_df["advertId"].nunique())

#unique_change_times = df_guten['advertId'].nunique()
#print(unique_change_times)

59


In [33]:
# Flatten the JSON data
flattened_data_guten = []
flattened_data_giper = []
flattened_data_kitchen = []
flattened_data_smart = []


for entry in all_campaign_data_guten:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_guten.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_giper:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_giper.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })

for entry in all_campaign_data_kitchen:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_kitchen.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                
for entry in all_campaign_data_smart:
    advertId = entry["advertId"]  # Extract the advertId
    for day in entry["days"]:
        date = day["date"]
        for app in day["apps"]:
            for nm in app["nm"]:
                flattened_data_smart.append({
                    "date": date,
                    "nmId": nm["nmId"],
                    "name": nm["name"],
                    "views": nm["views"],
                    "clicks": nm["clicks"],
                    "ctr": nm["ctr"],
                    "cpc": nm["cpc"],
                    "sum": nm["sum"],
                    "atbs": nm["atbs"],
                    "orders": nm["orders"],
                    "cr": nm["cr"],
                    "shks": nm["shks"],
                    "sum_price": nm["sum_price"],
                    "advertId": advertId  # Add advertId to each row
                })
                

# Create a DataFrame
df_guten = pd.DataFrame(flattened_data_guten)
df_giper = pd.DataFrame(flattened_data_giper)
df_kitchen = pd.DataFrame(flattened_data_kitchen)
df_smart = pd.DataFrame(flattened_data_smart)
# Convert the 'date' column to datetime
df_guten["date"] = pd.to_datetime(df_guten["date"])
df_giper["date"] = pd.to_datetime(df_giper["date"])
df_kitchen["date"] = pd.to_datetime(df_kitchen["date"])
df_smart["date"] = pd.to_datetime(df_smart["date"])
# Remove timezone information (make it timezone-naive)
df_guten["date"] = df_guten["date"].dt.tz_localize(None)
df_giper["date"] = df_giper["date"].dt.tz_localize(None)
df_kitchen["date"] = df_kitchen["date"].dt.tz_localize(None)
df_smart["date"] = df_smart["date"].dt.tz_localize(None)

# Display the DataFrame
df_guten

,date,nmId,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId
0,2025-03-01 03:00:00,181893203,Набор Fusion 5 ProGlide с 1 кассетой и подстав...,146,2,1.37,9.18,18.35,0,0,0.00,0,0,21498270
1,2025-03-01 03:00:00,181893214,Набор Fusion 5 ProGlide с гелем для бритья и п...,143,1,0.70,18.59,18.59,0,0,0.00,0,0,21498270
2,2025-03-01 03:00:00,181893183,Набор Fusion 5 ProGlide с 1 кассетой и чехлом ...,5,0,0.00,0.00,0.65,0,0,0.00,0,0,21498270
3,2025-03-01 03:00:00,181893193,Набор ProGlide Power бритва в дорожной косметичке,205,1,0.49,26.40,26.40,0,0,0.00,0,0,21498270
4,2025-03-01 03:00:00,181893223,Набор ProGlide Power бритва с 1 кассетой и гел...,3,0,0.00,0.00,0.12,0,0,0.00,0,0,21498270
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8475,2025-03-12 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,546,10,1.83,6.28,62.78,0,0,0.00,0,0,18870619
8476,2025-03-12 03:00:00,209076655,Яйцеварка электрическая автоматическая E 3,1022,33,3.23,3.71,122.31,2,1,3.03,1,3972,18870619
8477,2025-03-12 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,372,5,1.34,14.54,72.70,1,0,0.00,0,0,18870619
8478,2025-03-12 03:00:00,46292575,Яйцеварка электрическая автоматическая E 9,184,4,2.17,9.79,39.15,1,1,25.00,1,6992,18870619


In [34]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_guten.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [35]:
# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_guten = (
    df_guten.groupby([df_guten["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_giper = (
    df_giper.groupby([df_giper["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_kitchen = (
    df_kitchen.groupby([df_kitchen["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)

# Group by 'date' (day only) and 'nmId', summing numeric columns except 'advertId'
df_grouped_smart = (
    df_smart.groupby([df_smart["date"].dt.date, "nmId", "advertId"], as_index=False)
    .agg({
        "date": "first",  # Keep the first date (to retain the day)
        "name": "first",  # Keep the first name (or customize this logic)
        "views": "sum",
        "clicks": "sum",
        "ctr": "mean",  # Sum or average, depending on your needs
        "cpc": "mean",  # Use mean for 'cpc' (cost per click)
        "sum": "sum",
        "atbs": "sum",
        "orders": "sum",
        "cr": "mean",  # Use mean for 'cr' (conversion rate)
        "shks": "sum",
        "sum_price": "sum",
        "advertId": "first"  # Keep the first 'advertId' (no summing)
    })
)


# Rename the 'date' column to 'day' for clarity
df_grouped_guten.rename(columns={"date": "day"}, inplace=True)
df_grouped_giper.rename(columns={"date": "day"}, inplace=True)
df_grouped_kitchen.rename(columns={"date": "day"}, inplace=True)
df_grouped_smart.rename(columns={"date": "day"}, inplace=True)

df_grouped_guten['Project'] = 'WB-GutenTech'
df_grouped_giper['Project'] = 'WB-ГиперМаркет'
df_grouped_kitchen['Project'] = 'WB-KitchenAid'
df_grouped_smart['Project'] = 'WB-Smart-Market'

# Concatenate the DataFrames
df_grouped_combined_campaigns = pd.concat([df_grouped_guten, df_grouped_giper, df_grouped_kitchen, df_grouped_smart], ignore_index=True)
df_grouped_combined_campaigns['Marketplace'] = 'Wildberries'

# Display the grouped DataFrame
df_grouped_combined_campaigns

,nmId,day,name,views,clicks,ctr,cpc,sum,atbs,orders,cr,shks,sum_price,advertId,Project,Marketplace
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,49,1,1.043333,0.886667,4.10,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
1,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,362,17,4.586667,5.723333,74.22,2,0,0.0,0,0,21202745,WB-GutenTech,Wildberries
2,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,89,4,5.070000,1.653333,7.16,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
3,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,355,16,3.720000,6.093333,72.78,1,0,0.0,0,0,21202745,WB-GutenTech,Wildberries
4,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,111,3,1.586667,0.566667,8.76,0,0,0.0,0,0,21202453,WB-GutenTech,Wildberries
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,253299215,2025-03-12 03:00:00,Триммер для бороды и усов BT3421,251,21,8.910000,1.616667,25.01,3,0,0.0,0,0,19597465,WB-Smart-Market,Wildberries
5813,288465476,2025-03-12 03:00:00,Набор Fusion 5 ProGlide с 1 кассетой и чехлом ...,1,0,0.000000,0.000000,0.06,0,0,0.0,0,0,23236144,WB-Smart-Market,Wildberries
5814,313543155,2025-03-12 03:00:00,,11,0,0.000000,0.000000,0.54,0,0,0.0,0,0,23236221,WB-Smart-Market,Wildberries
5815,313543156,2025-03-12 03:00:00,,16,0,0.000000,0.000000,0.80,0,0,0.0,0,0,23236221,WB-Smart-Market,Wildberries


In [36]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_grouped_combined_campaigns.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [37]:
# Merge the grouped DataFrame with the filtered_df to add additional columns
df_final = df_grouped_combined_campaigns.merge(
    filtered_df[["advertId", "endTime", "createTime", "startTime","name", "status", "type"]],
    on="advertId",
    how="left"
)

# Drop the columns 'ctr', 'cpc', and 'cr'
df_final = df_final.drop(columns=["ctr", "cpc", "cr"])

# Rename the columns 'name_x' and 'name_y'
df_final.rename(
    columns={
        "name_x": "name_product",  # Rename 'name_x' to 'name_product'
        "name_y": "name_campaign"  # Rename 'name_y' to 'name_campaign'
    },
    inplace=True
)

# Display the final DataFrame
df_final
df_final_copy = df_final.copy()


In [38]:
df_final

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,advertId,Project,Marketplace,endTime,createTime,startTime,name_campaign,status,type
0,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,49,1,4.10,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
1,25623718,2025-03-01 03:00:00,Бритва мужская электрическая S3 300BT с триммером,362,17,74.22,2,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
2,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,89,4,7.16,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
3,25624340,2025-03-01 03:00:00,Электробритва Series 3 300s,355,16,72.78,1,0,0,0,21202745,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион
4,25628178,2025-03-01 03:00:00,Бритва мужская электрическая S3 300s,111,3,8.76,0,0,0,0,21202453,WB-GutenTech,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,253299215,2025-03-12 03:00:00,Триммер для бороды и усов BT3421,251,21,25.01,3,0,0,0,19597465,WB-Smart-Market,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания
5813,288465476,2025-03-12 03:00:00,Набор Fusion 5 ProGlide с 1 кассетой и чехлом ...,1,0,0.06,0,0,0,0,23236144,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания
5814,313543155,2025-03-12 03:00:00,,11,0,0.54,0,0,0,0,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания
5815,313543156,2025-03-12 03:00:00,,16,0,0.80,0,0,0,0,23236221,WB-Smart-Market,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания


In [39]:
# Specify the file name for the Excel file
file_name = "df_guten_data.xlsx"

# Export the DataFrame to an Excel file
df_final.to_excel(file_name, index=False)

print(f"DataFrame exported to {file_name} successfully.")

DataFrame exported to df_guten_data.xlsx successfully.


In [ ]:
# API endpoint
url = "https://seller-analytics-api.wildberries.ru/api/v2/nm-report/detail/history"

# Define headers and project names
projects = {
    'WB-GutenTech': headers_guten,
    'WB-ГиперМаркет': headers_giper,
    'WB-KitchenAid': headers_kitchen,
    'WB-Smart-Market': headers_smart
}

# Calculate yesterday's date
yesterday = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Define the period for the report (only yesterday)
period = {
    "begin": yesterday,
    "end": yesterday
}

# Function to send a single POST request
def send_request(nm_ids, period, headers):
    request_body = {
        "nmIDs": nm_ids,
        "period": period
    }
    response = requests.post(url, headers=headers, json=request_body)
    return response

# Function to fetch data in batches
def fetch_data_in_batches(nm_ids, period, headers):
    all_data = []
    batch_size = 20  # Maximum allowed nmIDs per request
    requests_per_minute = 3  # Maximum allowed requests per minute
    interval = 60 / requests_per_minute  # Time interval between requests in seconds

    for i in range(0, len(nm_ids), batch_size):
        batch = nm_ids[i:i + batch_size]
        response = send_request(batch, period, headers)

        if response.status_code == 200:
            try:
                data = response.json()
                if not data.get('error') and 'data' in data:
                    all_data.extend(data['data'])
                    print(f"Data retrieved successfully for batch {i // batch_size + 1}")
                else:
                    print(f"Error in response for batch {i // batch_size + 1}: {data.get('errorText', 'No error text')}")
            except ValueError as e:
                print(f"Failed to decode JSON for batch {i // batch_size + 1}: {e}")
        else:
            print(f"Error for batch {i // batch_size + 1}: {response.status_code}, {response.text}")

        time.sleep(interval)  # Respect the rate limit

    return all_data

# List to store DataFrames for each project
dataframes = []

# Process each project
for project_name, headers in projects.items():
    # Filter the DataFrame for the current project
    filtered_df = df_final[df_final['Project'] == project_name]
    unique_nmId_values = filtered_df['nmId'].unique().tolist()
    print(f"Total unique nmId values for {project_name}:", len(unique_nmId_values))  # Debugging line

    # Fetch data for all unique nmId values
    all_data = fetch_data_in_batches(unique_nmId_values, period, headers)

    # Flatten the nested 'history' data for easier analysis
    flattened_data = []
    for item in all_data:
        nmID = item['nmID']
        imtName = item['imtName']
        vendorCode = item['vendorCode']
        for history in item['history']:
            history_entry = {
                'nmID': nmID,
                'imtName': imtName,
                'vendorCode': vendorCode,
                'dt': history['dt'],
                'openCardCount': history['openCardCount'],
                'addToCartCount': history['addToCartCount'],
                'addToCartConversion': history['addToCartConversion'],
                'ordersCount': history['ordersCount'],
                'ordersSumRub': history['ordersSumRub'],
                'cartToOrderConversion': history['cartToOrderConversion'],
                'buyoutsCount': history['buyoutsCount'],
                'buyoutsSumRub': history['buyoutsSumRub'],
                'buyoutPercent': history['buyoutPercent']
            }
            flattened_data.append(history_entry)

    # Convert the flattened data to a DataFrame
    df_copy = pd.DataFrame(flattened_data)
    print(f"Data for {project_name}:\n", df_copy.head())

    # Add the DataFrame to the list
    dataframes.append(df_copy)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(dataframes, ignore_index=True)

# Display the combined DataFrame
print("Combined DataFrame:\n", combined_df.head())
df_copy=combined_df.copy()

Total unique nmId values for WB-GutenTech: 254
Data retrieved successfully for batch 1
Data retrieved successfully for batch 2
Data retrieved successfully for batch 3
Data retrieved successfully for batch 4
Data retrieved successfully for batch 5
Data retrieved successfully for batch 6
Data retrieved successfully for batch 7
Data retrieved successfully for batch 8
Data retrieved successfully for batch 9
Data retrieved successfully for batch 10
Data retrieved successfully for batch 11
Data retrieved successfully for batch 12
Data retrieved successfully for batch 13
Data for WB-GutenTech:
         nmID                                            imtName  \
0  110059376            Электрическая вафельница WA 3772 silber   
1  135577637  Сменные кассеты для бритья Fusion 5 ProGlide 2 шт   
2   43371059                      Кухонные весы Kitchen EcoMate   
3   25725047                   Триммер для бороды и усов BT3222   
4  133180387  Чайник электрический с регулировкой температур...   

  

In [58]:
df_copy=combined_df.copy()
df_final_copy=df_final.copy()

In [44]:
df_copy

,nmID,imtName,vendorCode,dt,openCardCount,addToCartCount,addToCartConversion,ordersCount,ordersSumRub,cartToOrderConversion,buyoutsCount,buyoutsSumRub,buyoutPercent
0,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-07,45,2,4,0,0,0,0,0,0
1,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-08,20,0,0,0,0,0,0,0,0
2,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-09,48,1,2,1,7340,100,1,7340,100
3,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-10,47,3,6,0,0,0,0,0,0
4,46292575,Яйцеварка электрическая автоматическая E 9,CM-00-00001075,2025-03-11,30,2,7,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3565,313543156,Сменные кассеты для бритья лезвия Fusion 5 Pro...,7702018085934,2025-03-09,86,17,20,6,30535,35,3,15267,100
3566,313543156,Сменные кассеты для бритья лезвия Fusion 5 Pro...,7702018085934,2025-03-10,56,11,20,2,10178,18,1,5089,100
3567,313543156,Сменные кассеты для бритья лезвия Fusion 5 Pro...,7702018085934,2025-03-11,53,10,19,2,10178,20,1,5089,100
3568,313543156,Сменные кассеты для бритья лезвия Fusion 5 Pro...,7702018085934,2025-03-12,29,8,28,5,25589,63,2,10250,100


In [45]:
df_unique = df_copy['nmID'].unique()
print(df_unique)

[ 46292575  25623718 110511380 133180387  25962484 110059376  25628178
 135577637  26025671  26026221 128291868  25624340  25725047 137609308
 133367874 121061623 133358869 133360169  43371059 137609254 137609782
 137609742 137609392 137609312 143478977 137609342 145265707 145266250
 144986300 143429326 138937204 137609695 137609384 137609383 137609340
 143493692 144980785 137609389 137609754 137609323 147720623 147720619
 159121839 156634286 159121843 155383929 145759203 156636098 155776833
 159121846 145813924 159121842 156634283 147720616 145813923 159121836
 145759198 147720621 159121837 145266634 159121859 159121850 159488075
 162402302 159121851 159121854 159121853 173292231 159121848 162402289
 159121858 171562370 159121849 161420838 170525397 159121847 159121857
 159121852 171562371 171562375 181893195 187556878 181893209 173292233
 188403472 200571555 200576962 181893214 173292235 181674546 198208484
 181893223 200580096 181893183 181893203 198084188 173292234 195507426
 18189

In [46]:
# Assuming your DataFrame is named df
specific_nmID = 137609342

# Filter the DataFrame for the specific nmID
filtered_df_id = df_copy[df_copy['nmID'] == specific_nmID]

# Group by nmID and sum the ordersSumRub
total_ordersSumRub = filtered_df_id['ordersSumRub'].sum()

print(f"Total ordersSumRub for nmID {specific_nmID}: {total_ordersSumRub}")

Total ordersSumRub for nmID 137609342: 4120


In [47]:
df_unique = df_copy['ordersSumRub'].sum()
print(df_unique)

10369916


In [59]:
# Convert 'day' and 'dt' to datetime for accurate merging
df_final_copy['day'] = pd.to_datetime(df_final_copy['day']).dt.date
df_copy['dt'] = pd.to_datetime(df_copy['dt']).dt.date

# Rename columns in df2 to match df1 for merging
df_copy.rename(columns={'nmID': 'nmId', 'dt': 'day'}, inplace=True)

# Merge the DataFrames on 'nmId', 'day', and 'Project'
merged_df_2 = pd.merge(
    df_final_copy,
    df_copy[['nmId', 'day', 'ordersCount', 'ordersSumRub','addToCartCount']],
    on=['nmId', 'day'],
    how='left'
)

# Fill NaN values with 0 for ordersCount and ordersSumRub
merged_df_2['ordersCount'].fillna(0, inplace=True)
merged_df_2['ordersSumRub'].fillna(0, inplace=True)
merged_df_2['addToCartCount'].fillna(0, inplace=True)


# Display the merged DataFrame
merged_df_2

C:\Users\User\AppData\Local\Temp\ipykernel_13752\76939134.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_df_2['ordersCount'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_13752\76939134.py:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For examp

,nmId,day,name_product,views,clicks,sum,atbs,orders,shks,sum_price,...,Marketplace,endTime,createTime,startTime,name_campaign,status,type,ordersCount,ordersSumRub,addToCartCount
0,25623718,2025-03-01,Бритва мужская электрическая S3 300BT с триммером,49,1,4.10,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0.0,0.0,0.0
1,25623718,2025-03-01,Бритва мужская электрическая S3 300BT с триммером,362,17,74.22,2,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0.0,0.0,0.0
2,25624340,2025-03-01,Электробритва Series 3 300s,89,4,7.16,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0.0,0.0,0.0
3,25624340,2025-03-01,Электробритва Series 3 300s,355,16,72.78,1,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2025-01-09,BRAUN_3 серия Электробритвы_07.11.2024,Идут показы,Аукцион,0.0,0.0,0.0
4,25628178,2025-03-01,Бритва мужская электрическая S3 300s,111,3,8.76,0,0,0,0,...,Wildberries,2100-01-01,2024-11-07,2024-11-07,BRAUN_Электробритвы_07.11.2024_РКП,Идут показы,Автоматическая кампания,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5812,253299215,2025-03-12,Триммер для бороды и усов BT3421,251,21,25.01,3,0,0,0,...,Wildberries,2100-01-01,2024-08-29,2025-02-20,Braun_Триммеры_22.10.2024,Идут показы,Автоматическая кампания,2.0,6886.0,4.0
5813,288465476,2025-03-12,Набор Fusion 5 ProGlide с 1 кассетой и чехлом ...,1,0,0.06,0,0,0,0,...,Wildberries,2100-01-01,2025-02-12,2025-02-24,Gillette_Бритвы + наборы_12.02.2025,Кампания на паузе,Автоматическая кампания,2.0,3450.0,7.0
5814,313543155,2025-03-12,,11,0,0.54,0,0,0,0,...,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,1.0,4796.0,5.0
5815,313543156,2025-03-12,,16,0,0.80,0,0,0,0,...,Wildberries,2100-01-01,2025-02-12,2025-02-21,Gillette_Кассеты_12.02.2025,Кампания на паузе,Автоматическая кампания,5.0,25589.0,8.0


In [49]:
df_unique = merged_df_2['ordersSumRub'].sum()
print(df_unique)


10866506.0


## Insert the data

In [60]:
password = os.getenv('ClickHouse')
# Define connection parameters
client = get_client(
    host='rc1a-vk5i3icccvmfk6cm.mdb.yandexcloud.net',  # Your Yandex Cloud ClickHouse host
    port=8443,                                          # Yandex Cloud uses port 8443 for HTTPS
    username='user1',                                      # Your ClickHouse username
    password= password,                           # Your ClickHouse password
    database='user1',                            # Your database name
    secure=True,                                        # Use HTTPS
    verify=False                                        # Disable SSL certificate verification 
    # Define the data to insert
)

In [61]:
# Ensure date columns are in the correct format for ClickHouse
merged_df_2['day'] = pd.to_datetime(merged_df_2['day'])  # Convert to datetime


# Debugging: Check the data types of the DataFrame
print("Data types of merged_df:")
print(merged_df_2.dtypes)

# Ensure the DataFrame has the correct columns
columns = [
    'nmId', 'day', 'name_product', 'views', 'clicks', 'sum', 'atbs', 'orders', 'shks',
    'sum_price', 'advertId', 'Project', 'Marketplace','endTime', 'createTime', 'startTime', 
    'name_campaign', 'status', 'type','ordersCount', 'ordersSumRub','addToCartCount'
]

# Reorder columns to match the expected order
merget_df_copy_2 = merged_df_2[columns]

# Convert DataFrame to a list of tuples for bulk insertion
data = [tuple(row) for row in merget_df_copy_2.to_numpy()]

# Debugging: Check the structure of the data
print("Sample data to insert:", data[:5])  # Print the first 5 rows to check the structure

# Define the table name
table_name = 'campaign_data_wb'

# Use the insert method for bulk insertion
client.insert(table_name, data, column_names=columns)
print("Data inserted successfully!")

Data types of merged_df:
nmId                       int64
day               datetime64[ns]
name_product              object
views                      int64
clicks                     int64
sum                      float64
atbs                       int64
orders                     int64
shks                       int64
sum_price                  int64
advertId                   int64
Project                   object
Marketplace               object
endTime                   object
createTime                object
startTime                 object
name_campaign             object
status                    object
type                      object
ordersCount              float64
ordersSumRub             float64
addToCartCount           float64
dtype: object
Sample data to insert: [(25623718, Timestamp('2025-03-01 00:00:00'), 'Бритва мужская электрическая S3 300BT с триммером', 49, 1, 4.1000000000000005, 0, 0, 0, 0, 21202453, 'WB-GutenTech', 'Wildberries', datetime.date(2100, 1, 1), dateti